In [1]:
from datetime import date
from pandas import json_normalize
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
df_work_raw = catalog.load('raw/openalex/work#parquet')

                    INFO     Loading data from raw/openalex/work#parquet (ParquetDataset)...    ]8;id=11069;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=171417;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

In [3]:
df_work_raw = df_work_raw.convert_dtypes()
df_work_raw = df_work_raw.loc[:,['id','locations']]

In [4]:
df_work_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1000 non-null   string
 1   locations  1000 non-null   object
dtypes: object(1), string(1)
memory usage: 15.8+ KB


In [5]:
df_work_raw

,id,locations
0,https://openalex.org/W2089852843,"[{'is_accepted': False, 'is_oa': False, 'is_pu..."
1,https://openalex.org/W3037375387,"[{'is_accepted': True, 'is_oa': True, 'is_publ..."
2,https://openalex.org/W2032143374,"[{'is_accepted': False, 'is_oa': False, 'is_pu..."
3,https://openalex.org/W2021071494,"[{'is_accepted': False, 'is_oa': False, 'is_pu..."
4,https://openalex.org/W2154507013,"[{'is_accepted': True, 'is_oa': True, 'is_publ..."
...,...,...
995,https://openalex.org/W2018147890,"[{'is_accepted': False, 'is_oa': False, 'is_pu..."
996,https://openalex.org/W2105314474,"[{'is_accepted': True, 'is_oa': True, 'is_publ..."
997,https://openalex.org/W1972746868,"[{'is_accepted': False, 'is_oa': False, 'is_pu..."
998,https://openalex.org/W1983613813,"[{'is_accepted': False, 'is_oa': False, 'is_pu..."


In [6]:
df_work_authorship = df_work_raw.explode('locations').reset_index(drop=True)

In [7]:
def openalex_load_work_location(df_work_raw):

    df_work_location = df_work_raw.explode('locations').reset_index(drop=True)

    df_work_location = pd.concat([df_work_location['id'], json_normalize(df_work_location['locations'])], axis=1)

    df_work_location.columns = df_work_location.columns.str.replace('.', '_')

    df_work_location = df_work_location[[
        'id', 
        'source_id', 'source_display_name', 'source_is_core', 'source_type',
        'source_host_organization', 'source_host_organization_name',
        'is_accepted', 'is_oa', 'is_published', 'landing_page_url',
        'license', 'license_id', 'pdf_url', 'version',
        # 'source_host_organization_lineage', 'source_host_organization_lineage_names', 'source_issn',
        'source_is_in_doaj', 'source_is_oa', 'source_issn_l'
    ]]

    df_work_location['load_datetime'] = date.today()

    return df_work_location


In [8]:
stage_work_location = openalex_load_work_location(df_work_raw)
stage_work_location

,id,source_id,source_display_name,source_is_core,source_type,source_host_organization,source_host_organization_name,is_accepted,is_oa,is_published,landing_page_url,license,license_id,pdf_url,version,source_is_in_doaj,source_is_oa,source_issn_l,load_datetime
0,https://openalex.org/W2089852843,https://openalex.org/S24688465,Physica A Statistical Mechanics and its Applic...,True,journal,https://openalex.org/P4310320990,Elsevier BV,False,False,False,https://doi.org/10.1016/s0378-4371(98)00437-3,None,None,None,None,False,False,0378-4371,2025-09-28
1,https://openalex.org/W3037375387,https://openalex.org/S143381477,Environment International,True,journal,https://openalex.org/P4310320990,Elsevier BV,True,True,True,https://doi.org/10.1016/j.envint.2020.105876,cc-by-nc-nd,https://openalex.org/licenses/cc-by-nc-nd,None,publishedVersion,True,True,0160-4120,2025-09-28
2,https://openalex.org/W3037375387,https://openalex.org/S4306401280,DOAJ (DOAJ: Directory of Open Access Journals),False,repository,None,None,False,False,False,https://doaj.org/article/9410c9207e9e450e8f655...,None,None,None,None,False,True,None,2025-09-28
3,https://openalex.org/W3037375387,https://openalex.org/S4306402455,CONICET Digital (CONICET),False,repository,https://openalex.org/I4210148385,Biomedicine Research Institute of Buenos Aires...,True,True,True,https://hdl.handle.net/11336/142501,cc-by-nc-nd,https://openalex.org/licenses/cc-by-nc-nd,https://ri.conicet.gov.ar/bitstream/11336/1425...,publishedVersion,False,True,None,2025-09-28
4,https://openalex.org/W3037375387,https://openalex.org/S4306525036,PubMed,False,repository,https://openalex.org/I1299303238,National Institutes of Health,False,False,False,https://pubmed.ncbi.nlm.nih.gov/32590284,None,None,None,None,False,False,None,2025-09-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2366,https://openalex.org/W1983613813,https://openalex.org/S4306525036,PubMed,False,repository,https://openalex.org/I1299303238,National Institutes of Health,False,False,False,https://pubmed.ncbi.nlm.nih.gov/22818787,None,None,None,None,False,False,None,2025-09-28
2367,https://openalex.org/W2002797015,https://openalex.org/S57491091,Archives of Virology,True,journal,https://openalex.org/P4310319900,Springer Science+Business Media,False,False,False,https://doi.org/10.1007/s00705-012-1302-2,None,None,None,None,False,False,0304-8608,2025-09-28
2368,https://openalex.org/W2002797015,https://openalex.org/S4306402455,CONICET Digital (CONICET),False,repository,https://openalex.org/I4210148385,Biomedicine Research Institute of Buenos Aires...,True,True,True,http://hdl.handle.net/11336/196680,cc-by-nc-sa,https://openalex.org/licenses/cc-by-nc-sa,https://ri.conicet.gov.ar/bitstream/11336/1966...,publishedVersion,False,True,None,2025-09-28
2369,https://openalex.org/W2002797015,https://openalex.org/S4306402641,LA Referencia (Red Federada de Repositorios In...,False,repository,https://openalex.org/I4383465926,LA Referencia,False,True,False,https://digital.cic.gba.gob.ar/handle/11746/7250,cc-by-nc,https://openalex.org/licenses/cc-by-nc,http://digital.cic.gba.gob.ar/bitstreams/5d578...,submittedVersion,False,True,None,2025-09-28
